In [75]:
import sys
print(sys.executable)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()
load_dotenv()

"""
Downloads data for every dat in date range for the given stock symbol. Saves it as parquet file in data/raw
"""
def dowload_one_file_of_raw_data(symbol, start_date, end_date):
    POLYGON_API_KEY = os.getenv('POLYGON_API_KEY')
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start_date}/{end_date}?apiKey={POLYGON_API_KEY}"
    response = requests.get(url)
    data = response.json()
    
    # Print keys to inspect the structure of the data
    print(dict(data).keys())

    # Extract the time series data
    time_series = data.get('results', [])
    
    # Check if time_series is empty or not
    if not time_series:
        print("No data available.")
        return

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(time_series)
    
    # convert 't' (timestamp) to datetime and rename it to 'Date'
    df['Date'] = pd.to_datetime(df['t'], unit='ms')
    df.set_index('Date', inplace=True)
    
    # drop the timestamp column 't'
    df.drop(columns=['t'], inplace=True)
    
    # rename columns for clarity
    df.rename(columns={
        'o': 'Open',
        'h': 'High',
        'l': 'Low',
        'c': 'Close',
        'v': 'Volume'
    }, inplace=True)
    print(df)
    

    path = f'../data/raw/rides_{start_date}-{end_date}.parquet'
    df.to_parquet(path, index=True)

# Example usage
dowload_one_file_of_raw_data(symbol='AAPL', start_date='2022-08-03', end_date='2024-08-03') 



/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python
dict_keys(['ticker', 'queryCount', 'resultsCount', 'adjusted', 'results', 'status', 'request_id', 'count'])
                          Volume        vw     Open   Close     High     Low  \
Date                                                                           
2022-08-04 04:00:00   55474144.0  165.5946  166.005  165.81  167.190  164.43   
2022-08-05 04:00:00   56696985.0  164.7432  163.210  165.35  165.850  163.00   
2022-08-08 04:00:00   60362338.0  165.8939  166.370  164.87  167.810  164.20   
2022-08-09 04:00:00   63075503.0  164.8395  164.020  164.92  165.820  163.25   
2022-08-10 04:00:00   70170540.0  168.3496  167.680  169.24  169.340  166.90   
...                          ...       ...      ...     ...      ...     ...   
2024-07-29 04:00:00   35153729.0  218.1319  216.960  218.24  219.300  215.75   
2024-07-30 04:00:00   40681625.0  218.4059  219.190  218.80  220.325  216.12   
2024-0

In [ ]:
# VALIDATION STEP